# Using a Calculator Tool with Claude
In this recipe, we'll demonstrate how to provide Claude with a simple calculator tool that it can use to perform arithmetic operations based on user input. We'll define the calculator tool and show how Claude can interact with it to solve mathematical problems.

## Step 1: Set up the environment

First, let's install the required libraries and set up the Anthropic API client.

In [ ]:
%pip install boto3

In [1]:
import os
import boto3

# "anthropic.claude-3-haiku-20240307-v1:0"  "anthropic.claude-3-sonnet-20240229-v1:0"  "anthropic.claude-3-opus-20240229-v1:0"
MODEL_ID = "anthropic.claude-3-opus-20240229-v1:0"

region_name= os.getenv("AWS_REGION", default="us-west-2")
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region_name)

## Step 2: Define the calculator tool
We'll define a simple calculator tool that can perform basic arithmetic operations. The tool will take a mathematical expression as input and return the result. 

Note that we are calling ```eval``` on the outputted expression. This is bad practice and should not be used generally but we are doing it for the purpose of demonstration.

In [2]:
import re

def calculate(expression):
    # Remove any non-digit or non-operator characters from the expression
    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    
    try:
        # Evaluate the expression using the built-in eval() function
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"

tool_spec = {
     "name": "calculator",
     "description": "A simple calculator that performs basic arithmetic operations.",
     "inputSchema": {
         "json": {
            "type": "object",
            "properties": {
                   "expression": {
                       "type": "string",
                       "description": "The mathematical expression to evaluate (e.g., '2 + 3 * 4')."
                    }
            },
             "required": ["expression"]
         }
    }
}

In this example, we define a calculate function that takes a mathematical expression as input, removes any non-digit or non-operator characters using a regular expression, and then evaluates the expression using the built-in eval() function. If the evaluation is successful, the result is returned as a string. If an error occurs during evaluation, an error message is returned.

We then define the calculator tool with an input schema that expects a single expression property of type string.

## Step 3: Interact with Claude
Now, let's see how Claude can interact with the calculator tool to solve mathematical problems.

In [3]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "calculator":
        return calculate(tool_input["expression"])

def chat_with_claude(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    
    converse_response = bedrock_client.converse(
        modelId=MODEL_ID,
        messages = [{
            "role": "user",
            "content": [{"text": user_message}]
        }],
        inferenceConfig = {"maxTokens": 4096},
        toolConfig = {
            'tools': [
                {
                    'toolSpec': tool_spec
                }
            ]
        }
    )
    message = converse_response['output']['message']

    print(f"\nInitial Response:")
    print(f"Stop Reason: {converse_response['stopReason']}")
    print(f"Content: {message['content']}")

    if converse_response['stopReason'] == "tool_use":
        tool_use = next(block['toolUse'] for block in message['content'] if block.get('toolUse'))
        tool_name = tool_use["name"]
        tool_input = tool_use["input"]

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input: {tool_input}")

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"Tool Result: {tool_result}")

        converse_response = bedrock_client.converse(
            modelId=MODEL_ID,
            inferenceConfig={"maxTokens": 4096},
            messages=[
                {"role": "user", "content": [{"text": user_message}]},
                {"role": "assistant", "content": message['content']},
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use['toolUseId'],
                                "content": [{"text": tool_result}],
                            }
                        }
                    ],
                },
            ],
            toolConfig={
                'tools': [
                    {
                        'toolSpec': tool_spec
                    }
                ]
            }
        )
        message = converse_response['output']['message']
    
    else:
        response = message


    final_response = next(
        (block['text'] for block in message['content'] if block.get('text')),
        None,
    )
    print(message['content'])
    print(f"\nFinal Response: {final_response}")

    return final_response

## Step 4: Try it out!

Let's try giving Claude a few example math questions now that it has access to a calculator.

In [4]:
chat_with_claude("What is the result of 1,984,135 * 9,343,116?")
chat_with_claude("Calculate (12851 - 593) * 301 + 76")
chat_with_claude("What is 15910385 divided by 193053?")


User Message: What is the result of 1,984,135 * 9,343,116?

Initial Response:
Stop Reason: tool_use
Content: [{'text': '<thinking>\nThe user is asking for the result of a mathematical expression. The calculator tool is the only relevant tool for this request. \nThe calculator tool requires an "expression" parameter. The user has directly provided the mathematical expression to evaluate in their request: 1,984,135 * 9,343,116.\nThe user has provided all the necessary information for the calculator tool, so I can proceed with calling the tool to get the answer to their question.\n</thinking>'}, {'toolUse': {'toolUseId': 'tooluse_TXrclbc6TaiRXpcDX7xY2g', 'name': 'calculator', 'input': {'expression': '1984135 * 9343116'}}}]

Tool Used: calculator
Tool Input: {'expression': '1984135 * 9343116'}
Tool Result: 18538003464660
[{'text': 'Therefore, the result of 1,984,135 * 9,343,116 is 18,538,003,464,660.'}]

Final Response: Therefore, the result of 1,984,135 * 9,343,116 is 18,538,003,464,660.

'Therefore, 15910385 divided by 193053 equals 82.41459599177428.'